In [1]:
#RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

In [2]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
pd.set_option('display.max_columns', 25)

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LassoCV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import confusion_matrix

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import statsmodels.api as sm
from statsmodels.api import OLS

import seaborn as sns
sns.set()

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

/Users/williamdrew/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/williamdrew/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/williamdrew/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/williamdrew/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/williamdrew/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/williamd

In [3]:
# https://stackoverflow.com/questions/30557409/spotify-api-post-call-response-415
client_id = "9cd3dd2ea2cf492ca28ab0247a79d781"
client_secret = "11c972ad002843e9be5ecc31f022dd6e"
grant_type = 'client_credentials'
body_params = {'grant_type' : grant_type}
url = 'https://accounts.spotify.com/api/token'

response = requests.post(url, data=body_params, auth = (client_id, client_secret)) 
print(response.content)

b'{"access_token":"BQCBEmShJncZn-motnYspdCP2AinmUkbsWFe6tywHDotGGNKW-8zX0EFF2sIxf5vsQEFQomFyjbu87esG7c","token_type":"Bearer","expires_in":3600,"scope":""}'


In [4]:
# Using Spotipy library
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

sp.audio_features(["0UaMYEvWZi0ZqiDOoHU3YI"])

[{'acousticness': 0.0311,
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0UaMYEvWZi0ZqiDOoHU3YI',
  'danceability': 0.904,
  'duration_ms': 226864,
  'energy': 0.813,
  'id': '0UaMYEvWZi0ZqiDOoHU3YI',
  'instrumentalness': 0.00697,
  'key': 4,
  'liveness': 0.0471,
  'loudness': -7.105,
  'mode': 0,
  'speechiness': 0.121,
  'tempo': 125.461,
  'time_signature': 4,
  'track_href': 'https://api.spotify.com/v1/tracks/0UaMYEvWZi0ZqiDOoHU3YI',
  'type': 'audio_features',
  'uri': 'spotify:track:0UaMYEvWZi0ZqiDOoHU3YI',
  'valence': 0.81}]

In [5]:
def songs_in_playlist(df, pid):
    '''
    input: dataframe and playlist ID
    returns: list of all songs on playlist by track_uri 
    '''
    playlist = np.where(df.pid == pid)
    return list(df.iloc[playlist]["track_uri"])

def playlist_means(tracks):
    '''
    input: list of tracks on a playlist (can be generated from songs_in_playlist())
    returns: statistics for audio features of playlist  
    '''
    num = len(tracks)
    # We can only pass 100 tracks at a time
    if num <= 100:
        p = pd.DataFrame.from_records(sp.audio_features(tracks))
    else:
        start = 0
        end = 100
        p = pd.DataFrame.from_records(sp.audio_features(tracks[start:end]))
        for i in range(int(num / 100)):
            start = end
            diff = num - end
            if diff > 100:
                end += 100
            else:
                end += diff
            
            new_p = pd.DataFrame.from_records(sp.audio_features(tracks[start:end]))
            p = p.append(new_p) 
    return p

---
**Note**

We could construct a model which determines if a user will like a certain playlist based on comparing their playlists and liked playlists to the audio features of our playlist. Train model on users playlists and likes, classify playlist as 0 or 1 if the user will like the playlist.

I believe our final model will look something like this: https://developer.spotify.com/console/get-recommendations/

In [6]:
songs = pd.read_pickle("pickles/songs_30k.pkl")
songs.head()

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.03110,https://api.spotify.com/v1/audio-analysis/0UaM...,0.904,226864,0.813,0UaMYEvWZi0ZqiDOoHU3YI,0.006970,4,0.0471,-7.105,0,0.1210,125.461,4,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,audio_features,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0.810
1,0.02490,https://api.spotify.com/v1/audio-analysis/6I9V...,0.774,198800,0.838,6I9VzXrHxO9rA9A5euc8Ak,0.025000,5,0.2420,-3.914,0,0.1140,143.040,4,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,audio_features,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,0.924
2,0.00238,https://api.spotify.com/v1/audio-analysis/0WqI...,0.664,235933,0.758,0WqIKmW4BTrj3eJFmnCKMv,0.000000,2,0.0598,-6.583,0,0.2100,99.259,4,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,audio_features,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,0.701
3,0.20200,https://api.spotify.com/v1/audio-analysis/1AWQ...,0.891,267267,0.714,1AWQoqb9bSvzTjaLralEkT,0.000234,4,0.0521,-6.055,0,0.1400,100.972,4,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,audio_features,spotify:track:1AWQoqb9bSvzTjaLralEkT,0.818
4,0.05610,https://api.spotify.com/v1/audio-analysis/1lzr...,0.853,227600,0.606,1lzr43nnXAijIGYnCT8M8H,0.000000,0,0.3130,-4.596,1,0.0713,94.759,4,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,audio_features,spotify:track:1lzr43nnXAijIGYnCT8M8H,0.654


In [7]:
songs2 = songs.copy()
songs2 = songs2.reset_index()
cols = ["acousticness", "danceability", "energy", "instrumentalness", "liveness", "speechiness", "tempo"]

def scaler(cols, df_old, df_new):
    for col in cols: 
        x_scaled = MinMaxScaler().fit_transform(df_old[col].reshape(-1,1))
        df_new[col] = x_scaled
    return(df_new)
        
songs2 = scaler(cols, songs, songs2)
songs2.head()

/Users/williamdrew/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  import sys


,index,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0,0.031225,https://api.spotify.com/v1/audio-analysis/0UaM...,0.912210,226864,0.813,0UaMYEvWZi0ZqiDOoHU3YI,0.006970,4,0.0471,-7.105,0,0.125129,0.505967,4,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,audio_features,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0.810
1,1,0.025000,https://api.spotify.com/v1/audio-analysis/6I9V...,0.781029,198800,0.838,6I9VzXrHxO9rA9A5euc8Ak,0.025000,5,0.2420,-3.914,0,0.117890,0.576860,4,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,audio_features,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,0.924
2,2,0.002390,https://api.spotify.com/v1/audio-analysis/0WqI...,0.670030,235933,0.758,0WqIKmW4BTrj3eJFmnCKMv,0.000000,2,0.0598,-6.583,0,0.217166,0.400298,4,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,audio_features,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,0.701
3,3,0.202811,https://api.spotify.com/v1/audio-analysis/1AWQ...,0.899092,267267,0.714,1AWQoqb9bSvzTjaLralEkT,0.000234,4,0.0521,-6.055,0,0.144778,0.407206,4,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,audio_features,spotify:track:1AWQoqb9bSvzTjaLralEkT,0.818
4,4,0.056325,https://api.spotify.com/v1/audio-analysis/1lzr...,0.860747,227600,0.606,1lzr43nnXAijIGYnCT8M8H,0.000000,0,0.3130,-4.596,1,0.073733,0.382150,4,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,audio_features,spotify:track:1lzr43nnXAijIGYnCT8M8H,0.654


In [9]:
song1 = songs2.iloc[np.where(songs.id == '0UaMYEvWZi0ZqiDOoHU3YI')]
song1

,index,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0,0.031225,https://api.spotify.com/v1/audio-analysis/0UaM...,0.91221,226864,0.813,0UaMYEvWZi0ZqiDOoHU3YI,0.00697,4,0.0471,-7.105,0,0.125129,0.505967,4,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,audio_features,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0.81
7734,73,0.031225,https://api.spotify.com/v1/audio-analysis/0UaM...,0.91221,226864,0.813,0UaMYEvWZi0ZqiDOoHU3YI,0.00697,4,0.0471,-7.105,0,0.125129,0.505967,4,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,audio_features,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0.81
14037,14,0.031225,https://api.spotify.com/v1/audio-analysis/0UaM...,0.91221,226864,0.813,0UaMYEvWZi0ZqiDOoHU3YI,0.00697,4,0.0471,-7.105,0,0.125129,0.505967,4,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,audio_features,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0.81
21536,42,0.031225,https://api.spotify.com/v1/audio-analysis/0UaM...,0.91221,226864,0.813,0UaMYEvWZi0ZqiDOoHU3YI,0.00697,4,0.0471,-7.105,0,0.125129,0.505967,4,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,audio_features,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0.81
24404,1,0.031225,https://api.spotify.com/v1/audio-analysis/0UaM...,0.91221,226864,0.813,0UaMYEvWZi0ZqiDOoHU3YI,0.00697,4,0.0471,-7.105,0,0.125129,0.505967,4,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,audio_features,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0.81
56659,51,0.031225,https://api.spotify.com/v1/audio-analysis/0UaM...,0.91221,226864,0.813,0UaMYEvWZi0ZqiDOoHU3YI,0.00697,4,0.0471,-7.105,0,0.125129,0.505967,4,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,audio_features,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0.81
72118,9,0.031225,https://api.spotify.com/v1/audio-analysis/0UaM...,0.91221,226864,0.813,0UaMYEvWZi0ZqiDOoHU3YI,0.00697,4,0.0471,-7.105,0,0.125129,0.505967,4,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,audio_features,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0.81
83454,60,0.031225,https://api.spotify.com/v1/audio-analysis/0UaM...,0.91221,226864,0.813,0UaMYEvWZi0ZqiDOoHU3YI,0.00697,4,0.0471,-7.105,0,0.125129,0.505967,4,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,audio_features,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0.81
90992,91,0.031225,https://api.spotify.com/v1/audio-analysis/0UaM...,0.91221,226864,0.813,0UaMYEvWZi0ZqiDOoHU3YI,0.00697,4,0.0471,-7.105,0,0.125129,0.505967,4,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,audio_features,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0.81
98238,78,0.031225,https://api.spotify.com/v1/audio-analysis/0UaM...,0.91221,226864,0.813,0UaMYEvWZi0ZqiDOoHU3YI,0.00697,4,0.0471,-7.105,0,0.125129,0.505967,4,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,audio_features,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0.81


In [128]:
def distance(idkey, songs):
    song = songs.iloc[np.where(songs.id == idkey)]
#     comp = []
#     for col in cols:
#         comp.append(np.subtract(songs[col], song[col])**2)
    dist = np.sqrt((np.subtract(songs.acousticness, song.acousticness))**2
                 + (np.subtract(songs.danceability, song.danceability))**2
                 + (np.subtract(songs.energy, song.energy))**2
                 + (np.subtract(songs.instrumentalness, song.instrumentalness))**2
                 + (np.subtract(songs.liveness, song.liveness))**2
                 + (np.subtract(songs.speechiness, song.speechiness))**2
                 + (np.subtract(songs.tempo, song.tempo))**2)
        
    return dist

In [129]:
top10_dist = distance('0WqIKmW4BTrj3eJFmnCKMv', songs2).sort_values()[0:10]

In [144]:
top10_dist.index

Int64Index([2, 374, 104, 45, 113, 370, 98, 257, 407, 21], dtype='int64')

In [131]:
top10 = songs2.iloc[top10_dist.index]

In [145]:
hrefs = top10['id']
hrefs

2      0WqIKmW4BTrj3eJFmnCKMv
374    7fYbFYt7X4FZvuJJC90EX0
104    4b1sQpvL7QVgamRZ74F1oA
45     1X5WGCrUMuwRFuYU1eAo2I
113    2MSjhkw47sGrjbZhColyVx
370    6FE2iI43OZnszFLuLtvvmg
98     2qWgqPdW1OiAP8KSBH1b93
257    6Xio661Y2iL7SDKFFOG9cv
407    3cdcT7I1xnQqQ84zMCtrKU
21     0uqPG793dkDDN7sCUJJIVC
Name: id, dtype: object

In [153]:
for ref in top10['id']:
    track = sp.track(ref)
    print(track['name'])
    print('by')
    for artist in track['artists']:
        print(artist['name'])
    print('-----------------------------------')

Crazy In Love (feat. Jay-Z)
by
Beyoncé
JAY Z
-----------------------------------
Problem
by
Ariana Grande
Iggy Azalea
-----------------------------------
Don't Recall
by
KARD
-----------------------------------
Livin on Sunday
by
Cris Cab
-----------------------------------
Aquaman
by
Jay Park
-----------------------------------
Classic
by
MKTO
-----------------------------------
FXXK WIT US
by
LEE HI
Dok2
-----------------------------------
Who Got Da Props
by
Black Moon
-----------------------------------
Hyperreal
by
Flume
KUČKA
-----------------------------------
Where Is The Love?
by
The Black Eyed Peas
-----------------------------------
